In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/human-canceling_video/'

/content/drive/MyDrive/human-canceling_video


In [3]:
!pip install imgcrop

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import sys
import cv2

def mp42img(video_path, output_path, basename, ext='jpg'):
  print("mp4 to img")
  cap = cv2.VideoCapture(video_path)
  if not cap.isOpened():
    return

  os.makedirs(output_path, exist_ok=True)
  if os.path.isfile(output_path + "/" + "frame_000.png"):
    print("file exists!!")
    return
  base_path = os.path.join(output_path, basename)

  digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

  n = 0
  while True:
    ret, frame = cap.read()
    if ret:
      cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
      n += 1
    else:
      return

def semantic_segmentation(filename, output_path):
  os.makedirs(output_path, exist_ok=True)

  input_image = Image.open(frame_path + '/' + filename).convert('RGB')
  preprocess = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0)

  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
  else:
    input_batch = input_batch.to('cpu')
    model.to('cpu')

  with torch.no_grad():
    outputs = model(input_batch)['out'][0]
  output_predictions = outputs.argmax(0)

  palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
  colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
  colors = (colors % 255).numpy().astype("uint8")
  
  for i in range(21):
    colors[i][0] = 0
    colors[i][1] = 0
    colors[i][2] = 0

  #person = 15
  colors[15][0] = 255
  colors[15][1] = 255
  colors[15][2] = 255

  r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
  r.putpalette(colors)

  if r.mode != "RGB":
    r = r.convert("RGB")
  r.save(output_path + "/" + file_name)

# -------------------------------------------------------------------------------------------

frame_path = './output/frames'
mask_path  = './output/mask'
mp42img('./input/original_19201080.mp4', frame_path, 'frame', 'png')

model = torch.hub.load('pytorch/vision:v0.9.0', 'deeplabv3_resnet101', pretrained=True)
model.eval()
for file_name in tqdm(os.listdir(frame_path)):
  semantic_segmentation(file_name, mask_path)

mp4 to img


In [ ]:
!python test.py \
  --baseroot './output/frames/' \
  --baseroot_mask './output/mask/' \
  --results_path './output/results' \
  --gan_type 'WGAN' \
  --gpu_ids '1' \
  --epoch 40 \
  --batch_size 1 \
  --num_workers 8 \
  --pad_type 'zero' \
  --activation 'elu' \
  --norm 'none'

Generator is created!
Initialize generator with xavier type
-------------------------Loading Pretrained Model-------------------------
-------------------------Pretrained Model Loaded-------------------------
The overall number of images equals to 699
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3658: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import sys
import cv2

def img2mov(filename, input_path):
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    video = cv2.VideoWriter(filename, fourcc, 30.0, (1920, 1080))

    if not video.isOpened():
        print("can't be opened")
        sys.exit()
    
    dir_list = len(os.listdir(input_path))
    for i in tqdm(range(dir_list)):
        img = cv2.imread(input_path + "/" + str(i) + '.png')

        if img is None:
            print("can't read")

        video.write(img)

    video.release()

# -------------------------------------------------------------------------------------------

img2mov("shibuya/shibuya_19201080.mp4", "./shibuya/results")

  0%|          | 3/699 [00:00<00:35, 19.39it/s]

can't read


100%|██████████| 699/699 [00:50<00:00, 13.86it/s]
